In [72]:
import duckdb
import os
import glob
from datetime import datetime, timezone

In [73]:

# ---- CONFIG ----
## vigyazz a pathra
DB_PATH = "../databases/currency_rates.duckdb"
JSON_FOLDER = "../jsons/fx_jsons"
SCHEMA_NAME = "analytics"
TABLE_NAME = "int_currency_rates"

# 1) DB csatlakozás -- ha nincs meg db file akkor letrehoz egyet
con = duckdb.connect(DB_PATH)

In [59]:
# 8) Teszt folyt
df = con.execute("""
    with dates as (

        select distinct rate_date
        from analytics.int_currency_rates

    ),

    ordered as (

        select
            rate_date,
            lag(rate_date) over (order by rate_date) as prev_date
        from dates

    ),

    gaps as (

        select
            rate_date,
            prev_date,
            rate_date - prev_date as gap_days
        from ordered
        where prev_date is not null
        and rate_date - prev_date > 1

    )

    select *
    from gaps
""").df()

df.tail(50)

,rate_date,prev_date,gap_days


In [77]:
df = con.execute(
    """
    select
        *
    from analytics.int_currency_rates
    where rate_date in ('2025-12-05', '2025-12-04')
    """
).df()

df.head(20)

,filename,loaded_at_utc,rate_date,currency_pair,rate
0,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDZWL,321.999592
1,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-04,USDZWL,321.999592
2,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDZMW,23.111058
3,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-04,USDZMW,22.979795
4,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDZMK,9001.203584
5,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-04,USDZMK,9001.202424
6,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDZAR,16.926304
7,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-04,USDZAR,16.991595
8,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDYER,238.550363
9,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-04,USDYER,238.395332


In [78]:
# futasd a checkpointot, h a currency_rates.duckdb.wal file (Write ahead log) ami a valtoztatasokat tartalömazza eltunjön
# vegul zard be a connectiont!
con.execute("CHECKPOINT;")
con.close()